<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

Импортируем необходимые библиотеки, затем считаем датасет в переменную data и изучим данные:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.utils import shuffle

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
display(data.head(10))
display(data.info())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


None

Поле Tenure содержит пропуски, проверим количество NaN в этом столбце и заполним пропуски нулями:

In [3]:
display(data['Tenure'].isna().sum())
data['Tenure'] = data['Tenure'].fillna(0)

909

Первые три столбца не должны использоваться для обучения модели, поэтому их мы исключим из датасета:

In [4]:
data = data.loc[:, 'CreditScore':'Exited']
display(data.head(10))

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


Разделим данные на тренировочную, валидационную и тестовую выборку в пропорции 3:1:1.

In [5]:
target = data['Exited']
features = data.drop('Exited', axis=1)
features_train, features_vt, target_train, target_vt = train_test_split(features, 
    target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_vt, 
    target_vt, test_size=0.5, random_state=12345)

Перед обучением модели необходимо сделать дополнительные преобразования: 
1. Категориальные данные перекодировать
2. Численные признаки масштабировать

In [6]:
categorial = ['Geography', 'Gender']
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

encoder = OrdinalEncoder()
features_train[categorial] = encoder.fit_transform(features_train[categorial])
features_valid[categorial] = encoder.transform(features_valid[categorial])
features_test[categorial] = encoder.transform(features_test[categorial])

scaler = StandardScaler()
features_train[numeric] = scaler.fit_transform(features_train[numeric])
features_valid[numeric] = scaler.fit_transform(features_valid[numeric])
features_test[numeric] = scaler.fit_transform(features_test[numeric])
display(features_train.head())
display(features_valid.head())
display(features_test.head())

/tmp/ipykernel_77/3295652780.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[categorial] = encoder.fit_transform(features_train[categorial])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/tmp/ipykernel_77/3295652780.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
7479,-0.886751,2.0,1.0,-0.373192,1.104696,1.232271,-0.891560,1,0,-0.187705
3411,0.608663,0.0,0.0,-0.183385,1.104696,0.600563,-0.891560,0,0,-0.333945
6027,2.052152,1.0,1.0,0.480939,-0.503694,1.027098,0.830152,0,1,1.503095
1247,-1.457915,0.0,1.0,-1.417129,0.461340,-1.233163,0.830152,1,0,-1.071061
3716,0.130961,1.0,0.0,-1.132419,-0.825373,1.140475,-0.891560,0,0,1.524268


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
8532,-0.643740,0.0,0.0,-0.366305,-0.775630,-1.235001,0.772118,1,0,-0.043299
5799,-0.229103,0.0,0.0,0.567908,-0.452720,-1.235001,-0.936108,1,1,1.439245
5511,0.206266,1.0,1.0,-0.646569,-1.421451,0.421977,-0.936108,1,0,-1.391276
7365,-0.819960,2.0,0.0,-0.272884,1.807651,1.216012,-0.936108,1,1,-0.812386
7367,-0.426055,2.0,1.0,0.287644,1.807651,-1.235001,0.772118,1,0,1.326385


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
7041,-2.152343,0.0,1.0,-0.088115,-0.836335,-1.195546,0.777708,1,0,0.639115
5709,-0.044761,0.0,0.0,0.010780,1.400282,-1.195546,-0.948613,1,0,-1.722312
7117,-0.863239,2.0,1.0,-0.780381,0.122215,0.734471,-0.948613,1,1,-1.426223
7775,-0.208456,2.0,1.0,0.109675,1.719798,-1.195546,0.777708,1,0,0.053240
8735,0.814642,0.0,1.0,-0.879276,1.719798,0.629177,-0.948613,0,1,-1.120569


## Исследование задачи

Проверим баланс классов. 

In [7]:
display(data['Exited'].sum()/len(data['Exited']))

0.2037

In [8]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predicted_values = model.predict(features_valid)
display("F1:",f1_score(predicted_values, target_valid))

'F1:'

0.2671614100185529

Положительных ответов ~20%, это может оказывать влияние на результат, поэтому применим несколько способов балансировки классов.
Метрика F1 для несбалансированных данных ~0.27. Это довольно низкий результат. В ходе альнейшего анализа мы проверим влияние разных методов балансировки классов и гиперпараметров модели на этот показатель.

## Борьба с дисбалансом

Попробуем использовать гиперпараметр class_weight для разых моделей:

In [9]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_values = model.predict(features_valid)
display("F1:", f1_score(predicted_values, target_valid))

'F1:'

0.4762711864406779

In [10]:
model = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
model.fit(features_train, target_train)
predicted_values = model.predict(features_valid)
display("F1:", f1_score(predicted_values, target_valid))

'F1:'

0.4760705289672544

Попробуем использовать гиперпараметр class_weight для модели случайного леса и пиграться с другими гиперпараметрами, чтобы найти лучший результат:

In [11]:
for depth in range(1, 16, 1):
    model = RandomForestClassifier(n_estimators=40, max_depth=depth, random_state=12345, class_weight='balanced')
    model.fit(features_train, target_train)
    predicted_values = model.predict(features_valid)
    print(depth, f1_score(predicted_values, target_valid))
    
for est in range(10, 110, 10):
    model = RandomForestClassifier(n_estimators=est, max_depth=9, random_state=12345, class_weight='balanced')
    model.fit(features_train, target_train)
    predicted_values = model.predict(features_valid)
    print(est, f1_score(predicted_values, target_valid))

1 0.5381083562901745
2 0.5761078998073218
3 0.5796545105566219
4 0.6068548387096774
5 0.6081632653061224
6 0.6136595310907238
7 0.6193001060445388
8 0.6153846153846154
9 0.6195899772209568
10 0.6058394160583942
11 0.5956354300385109
12 0.5861601085481682
13 0.5706293706293706
14 0.5693848354792561
15 0.5748148148148149
10 0.5900900900900902
20 0.6104328523862376
30 0.6174801362088536
40 0.6195899772209568
50 0.612756264236902
60 0.61345496009122
70 0.6153846153846155
80 0.6129032258064517
90 0.6189376443418014
100 0.6183908045977011


Попробуем технику апсемплинга и даунсемплинга:

In [12]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 3)

model = RandomForestClassifier(n_estimators=est, max_depth=9, random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.6219081272084807


In [13]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.4)

model = RandomForestClassifier(n_estimators=40, max_depth=9, random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.6272189349112426


Попробуем технику управления пороговым значением:

In [14]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.1, 0.4, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    print(round(threshold, 2), f1_score(target_valid, predicted_valid))

0.1 0.41917502787068
0.15 0.45236463331048665
0.2 0.47215295095594345
0.25 0.4827586206896552
0.3 0.46347031963470314
0.35 0.42988204456094364
0.4 0.3713850837138509


In [15]:
model = RandomForestClassifier(n_estimators=40, max_depth=9, random_state=12345)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.1, 0.4, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    print(round(threshold, 2), f1_score(target_valid, predicted_valid))

0.1 0.47878404053198226
0.15 0.5328124999999999
0.2 0.6064638783269962
0.25 0.6345733041575493
0.3 0.6070991432068543
0.35 0.6050198150594451
0.4 0.5929078014184397


Лучший результат показала модель случайного леса, с количеством деревьев 40 и максимальной глубиной 9. Лучшая техника балансировки классов: выбор порога, модельно показала точность F1 ~0.63, сочетание этих выводов будет использоваться для финального тестирования модели.

## Тестирование модели

In [16]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.3)

model = RandomForestClassifier(n_estimators=30, max_depth=8, random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_test = model.predict(features_test)

print("F1:", f1_score(target_test, predicted_test))
print("AUC-ROC:", roc_auc_score(target_test, predicted_test))

F1: 0.596923076923077
AUC-ROC: 0.7612195703305945


В результате получилась модель, которая показывает результат ~0.60 на тестовой выборке.
Метрика AUC-ROC равна ~0.76

В результате исследования была построена модель, способная предсказать отток клиентов банка. Итоговая модель показала результаты F1: 0.60 и AUC-ROC 0.76, что выполняет поставленную задачу.
Чтобы добиться этого результата были проделаны следующие шаги:
1. Обработаны данные, категорийный признако кодированы, а числовые масштабированы.
2. Был исследован дисбаланс классов.
3. Были рассмотрены разные модели, протестированы гиперпараметры и методы балансировки классов.
4. Выбрана лучшая модель, на которой был сделан итоговый расчет.